In [1]:
from extractors import *
from encoders import *
from general import *
from engine import *

d:\workspace\Hackathons\DB_All-Russian_hackathon_2024\.conda\lib\site-packages\pyannote\audio\core\io.py:43: UserWarning: torchaudio._backend.set_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  torchaudio.set_audio_backend("soundfile")
d:\workspace\Hackathons\DB_All-Russian_hackathon_2024\.conda\lib\site-packages\pyannote\audio\pipelines\speaker_verification.py:43: UserWarning: torchaudio._backend.get_audio_backend has been deprecated. With dispatcher enabled, this function is no-op. You can remove the function call.
  backend = torchaudio.get_audio_backend()
d:\workspace\Hackathons\DB_All-Russian_hackathon_2024\.conda\lib\inspect.py:869: UserWarning: Module 'speechbrain.pretrained' was deprecated, redirecting to 'speechbrain.inference'. Please update your script. This is a change from SpeechBrain 1.0. See: https://github.com/speechbrain/speechbrain/releases/tag/v1.0.0
  if ismodule(module) and hasattr(module, '__f

In [2]:
video_pipeline = Pipeline.from_yaml("video_pipeline.yaml")
audio_pipeline = Pipeline.from_yaml("audio_pipeline.yaml")
text_pipeline = Pipeline.from_yaml("text_pipeline.yaml")

d:\workspace\Hackathons\DB_All-Russian_hackathon_2024\.conda\lib\site-packages\huggingface_hub\file_download.py:1142: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


No language specified, language will be first be detected for each audio file (increases inference time).


Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.4.0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint C:\Users\murka\.cache\torch\whisperx-vad-segmentation.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.4.1. Bad things might happen unless you revert torch to 1.x.


In [3]:
%%time
print(video_pipeline("0c630520-6c3c-4948-a196-22a458a9cbdf.mp4").shape)
print(audio_pipeline("0c630520-6c3c-4948-a196-22a458a9cbdf.mp4").shape)
print(text_pipeline("0c630520-6c3c-4948-a196-22a458a9cbdf.mp4").shape)

d:\workspace\Hackathons\DB_All-Russian_hackathon_2024\.conda\lib\site-packages\timm\models\vision_transformer.py:92: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  x = F.scaled_dot_product_attention(


(21, 768)
(2241, 768)


d:\workspace\Hackathons\DB_All-Russian_hackathon_2024\.conda\lib\site-packages\pyannote\audio\utils\reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(


(1, 768)
CPU times: total: 906 ms
Wall time: 3.72 s


In [3]:
engine = QdrantEngine(url=":memory:", image_dim=768, audio_dim=768, text_dim=768)

In [4]:
from pathlib import Path
video_dir = Path(r"D:\workspace\Hackathons\ЦП Всерос 2024 доп данные\downloaded_dataset")

In [5]:
import pandas as pd
df = pd.read_csv(video_dir.parent.joinpath("train.csv"))

In [6]:
data = pd.DataFrame({})

In [7]:
image_empty_vec = torch.zeros((1, 768))
audio_empty_vec = torch.zeros((1, 768))
text_empty_vec = torch.zeros((1, 768))
from tqdm.notebook import tqdm
for i, row in tqdm(df.iterrows()):
    video_id = row["uuid"]
    duplicate = row["duplicate_for"]
    filepath = video_dir / f"{video_id}.mp4"
    if not filepath.exists():
        continue
    
    video_embedding = video_pipeline(filepath)

    try:
        audio_embedding = audio_pipeline(filepath)
        text_embedding = text_pipeline(filepath)
        results = engine.search(image_embedding=video_embedding, audio_embedding=audio_embedding, text_embedding=text_embedding).points
    except Exception as e:
        audio_embedding = audio_empty_vec.numpy()
        text_embedding = text_empty_vec.numpy()
        results = engine.search(image_embedding=video_embedding).points


    if len(results) != 0:
        duplicate_for = results[0].id
    else:
        duplicate_for = None

    data = pd.concat([data, pd.DataFrame({"uuid": [video_id], "answer": [duplicate], "predict": duplicate_for})])
    

    engine.add(video_id, video_embedding, audio_embedding, text_embedding) 


0it [00:00, ?it/s]

d:\workspace\Hackathons\DB_All-Russian_hackathon_2024\.conda\lib\site-packages\timm\models\vision_transformer.py:92: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  x = F.scaled_dot_product_attention(
d:\workspace\Hackathons\DB_All-Russian_hackathon_2024\.conda\lib\site-packages\pyannote\audio\utils\reproducibility.py:74: ReproducibilityWarning: TensorFloat-32 (TF32) has been disabled as it might lead to reproducibility issues and lower accuracy.
It can be re-enabled by calling
   >>> import torch
   >>> torch.backends.cuda.matmul.allow_tf32 = True
   >>> torch.backends.cudnn.allow_tf32 = True
See https://github.com/pyannote/pyannote-audio/issues/1370 for more details.

  warnings.warn(


No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No active speech found in audio
No activ

KeyboardInterrupt: 

In [17]:
data["answer"][pd.isna(data["answer"])] = ""
data["answer"][data["answer"] == None] = ""
data["predict"][data["predict"] == None] = ""

C:\Users\murka\AppData\Local\Temp\ipykernel_20096\2181735252.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data["predict"][data["predict"] == None] = ""


In [19]:
data[(data["answer"] == data["predict"])]

,uuid,answer,predict
0,5eb4127e-5694-492b-963c-6688522e9ad2,3726bb2d-3323-41f8-8eb2-0d7cf095d62b,3726bb2d-3323-41f8-8eb2-0d7cf095d62b
0,b5f191e6-42e0-43f5-8773-560643de17fb,314d2988-eb85-4581-8416-da998e036afe,314d2988-eb85-4581-8416-da998e036afe
0,025ee26a-7391-4f60-878a-7fc1928a967b,0fd09c1b-e19e-4b6e-84e6-35f9d4fc6f72,0fd09c1b-e19e-4b6e-84e6-35f9d4fc6f72
0,a18324cf-b2ad-41e2-86b8-e6923c5fdc36,2da574f2-5ae7-4d85-9f5b-c2970a315c1c,2da574f2-5ae7-4d85-9f5b-c2970a315c1c
0,2253aaa4-b29c-4b7d-b9cc-9286d23c44e8,3a20cbaf-798a-4b25-b2c1-e41e1c6ecd09,3a20cbaf-798a-4b25-b2c1-e41e1c6ecd09


In [9]:
text_embedding

(1, 768)

In [35]:
results = engine.search(image_embedding=video_embedding, audio_embedding=audio_embedding, text_embedding=text_embedding, image_threshold=0, audio_threshold=0, text_threshold=0.0).points

In [34]:
np.array(results[0].vector["text"]).shape

IndexError: list index out of range

In [22]:
text_embedding.shape

(1, 768)

In [36]:
results[0].score

0.0

In [45]:
from qdrant_client.local.distances import calculate_distance

In [51]:
calculate_distance(np.array(text_embedding), np.array(results[0].vector["text"]), distance_type="Dot") 

array([[0.]])

In [56]:
score(text_embedding, text_embedding)

1.0000001192092896

In [53]:
np.array(results[0].vector["text"])

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [18]:
text_pipeline(filepath)

array([[ 0.05112501,  0.03272877, -0.01156211,  0.02451173,  0.0048438 ,
        -0.05067518, -0.05085806, -0.01945942,  0.03010208,  0.00364512,
        -0.01376829,  0.01033569,  0.14470768,  0.03599226, -0.00491531,
        -0.04788486,  0.00196302, -0.01269231,  0.02729968,  0.00538157,
         0.01452344, -0.0390172 ,  0.04529845, -0.01999958,  0.04451223,
        -0.07013192,  0.00622712,  0.02947093,  0.01734442,  0.02616927,
         0.03230561, -0.0026346 ,  0.02514032,  0.02524344,  0.03243798,
         0.05989987, -0.01450588, -0.03884324,  0.02313459,  0.0354504 ,
         0.03174134,  0.03017657,  0.00680438, -0.04774344, -0.01115942,
        -0.02369665,  0.02711844,  0.03586359, -0.01492727, -0.02515883,
         0.00962217,  0.00163898,  0.00414406,  0.00840999, -0.04759841,
        -0.01996294,  0.00633833, -0.00510179, -0.06633098,  0.04771431,
         0.00096954,  0.04034598, -0.00515697, -0.00831401, -0.01009965,
        -0.00321905,  0.00375123, -0.01690102, -0.0

In [17]:
results[0].vector["text"]

[[0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,
  0.0,

In [9]:
filepath

WindowsPath('D:/workspace/Hackathons/ЦП Всерос 2024 доп данные/downloaded_dataset/15a8c00b-461e-407a-8b90-977e516208e8.mp4')

In [13]:
import qdrant_client

In [32]:
import qdrant_client.local
import qdrant_client.local.distances
import qdrant_client.local.multi_distances


qdrant_client.local.multi_distances.calculate_multi_distance(video_embedding, np.array([results[0].vector["image"]]), distance_type="Cosine")

array([37.00000108])

In [33]:
qdrant_client.local.multi_distances.calculate_multi_distance(audio_embedding, np.array([results[0].vector["audio"]]), distance_type="Cosine")

array([2738.99999944])

In [35]:
type(results[0].vector["image"][0])

list

In [30]:
np.array(results[0].vector["image"]).shape

(37, 768)

In [8]:
audio_embedding.shape

(0,)

In [12]:
for i in AudioNSecondsSplit(15)(filepath.as_posix()):
    print(i["raw_audio"].shape)

torch.Size([1, 661500])
torch.Size([1, 1028])


In [11]:
filepath

WindowsPath('D:/workspace/Hackathons/ЦП Всерос 2024 доп данные/downloaded_dataset/15a8c00b-461e-407a-8b90-977e516208e8.mp4')

In [9]:
# data["vid_coef"] = data["score"] / data["frames"]
data["aud_coef"] = data["score"] / data["audio_length"]

In [11]:
data.sort_values(by=["aud_coef"])

,uuid,frames,audio_length,duplicate_for,similar,score,aud_coef
0,245ad1d6-cc96-4785-af1a-ebc569f2adf8,1,249,NaN,406b8550-33a5-4853-8227-e0cd1db9abfa,205.060837,0.823537
0,12de4e4d-647b-4f4f-a8ba-1c821f99cac5,8,1992,NaN,8285f93d-c1b5-4b31-8a1a-4e94d06b5a99,1705.846994,0.856349
0,3dd424ce-f0e5-4727-88a3-8f318b612afd,12,2739,NaN,2147b7ce-0a90-45b4-8cf2-aa4ea723e54b,2346.691272,0.856769
0,3ddc9cf4-6534-49e6-add5-b6a5b3b3faa3,5,1743,NaN,12de4e4d-647b-4f4f-a8ba-1c821f99cac5,1499.236154,0.860147
0,2d2cde58-263d-467d-b163-af36beb62700,7,1245,NaN,2271dda7-763a-48be-b05f-9b06ca88e14a,1081.673626,0.868814
0,04395867-c740-4374-83aa-88546d40457f,2,498,NaN,3b447013-9202-415d-ae15-b1d76ce8e060,432.929818,0.869337
0,139a2a32-8ea1-4eae-9a0d-d9534e1dfd88,5,1494,NaN,31cc33d5-95de-4799-ad01-87c8498d1bde,1307.193075,0.874962
0,d444f2d0-a7cd-4c9b-bc56-8d5ef88ec015,6,1743,NaN,1e9efc51-a74c-4f32-b03e-71905f8d6dd1,1535.325533,0.880852
0,3e0e425f-9c4e-4f6e-a618-500e0883ec4d,2,249,NaN,10f67be6-7eb3-4288-9731-bb7013c4684a,226.431323,0.909363
0,025ee26a-7391-4f60-878a-7fc1928a967b,19,1992,0fd09c1b-e19e-4b6e-84e6-35f9d4fc6f72,0fd09c1b-e19e-4b6e-84e6-35f9d4fc6f72,1927.959740,0.967851


In [10]:
data.to_csv("bench.csv")